In [1]:
#Passo1:Importar bibliotecas
import requests
from bs4 import BeautifulSoup
import pandas as pd 
import time
import urllib.parse
import json
from concurrent.futures import ThreadPoolExecutor, as_completed
import random

# --- PASSO 2: Definir a URL e fazer a requisição inicial ---
url = "https://www.drogariaveracruz.com.br/medicamentos/"
url_base = url

# Header para evitar bloqueio
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# Criar sessão para reaproveitar conexões
session = requests.Session()
session.headers.update(headers)

# --- Funções auxiliares ---

def achar_nome(div):
    tag_h2 = div.find('h2', class_='title')
    if tag_h2:
        tag_a = tag_h2.find('a')
        if tag_a:
            return tag_a.get_text(strip=True)
    return "Nome não encontrado"

def achar_preco(div):
    tag_preco = div.find('p', class_="unit-price p-0")
    if tag_preco:
        return tag_preco.get_text().strip()
    return None

def achar_precopix(div):
    tag_precopix = div.find('p', class_="sale-price-pix")
    if tag_precopix:
        strong_tag = tag_precopix.find('strong')
        if strong_tag:
            return strong_tag.get_text(strip=True)
    return None 

def achar_total_paginas(soup_inicial):
    container_template = soup_inicial.find('div', class_='page-template')
    if container_template:
        divs_candidatas = container_template.find_all('div', class_='text-center pt-3')
        for div in divs_candidatas:
            texto = div.get_text(strip=True)
            if "Página" in texto:
                partes = texto.split()
                ultima_palavra = partes[-1]
                if ultima_palavra.isdigit():
                    return int(ultima_palavra)
    return 1

def achar_link(produto_html, url_base):
    tag_h2 = produto_html.find('h2', class_='title')
    if tag_h2:
        tag_a = tag_h2.find('a')
        if tag_a and 'href' in tag_a.attrs:
            link_relativo = tag_a.get('href')
            return urllib.parse.urljoin(url_base, link_relativo)
    return "Não encontrado"

def achar_precodesconto(produto_html):
    tag_venda = produto_html.find('p', class_="sale-price p-0")
    if tag_venda:
        strong_tag = tag_venda.find('strong')
        if strong_tag:
            return strong_tag.get_text(strip=True)
    return None

def extrair_detalhes_do_json(soup_detalhes):
    brand = "Não encontrada"
    code = "Não encontrado"
    script_tag = soup_detalhes.find('script', type='application/ld+json')
    if script_tag:
        try:
            raw_json = script_tag.string.strip()
            raw_json = raw_json.replace("\n", "").replace("\r", "").replace("\t", "")
            dados_json = json.loads(raw_json)
            if isinstance(dados_json, list):
                for item in dados_json:
                    if isinstance(item, dict) and 'brand' in item:
                        dados_json = item
                        break
            if 'brand' in dados_json and isinstance(dados_json['brand'], dict):
                brand = dados_json['brand'].get('name', "Não encontrada")
            code = dados_json.get('gtin13', "Não encontrado")
        except Exception as e:
            print("  -> Aviso: Erro ao ler JSON-LD:", e)
    return brand, code

def limpar_preco(preco_str):
    if not preco_str: return None
    try:
        return float(preco_str.replace("R$", "").strip().replace('.', '').replace(',', '.'))
    except (ValueError, TypeError):
        return None

# --- NOVO: baixar página com retry ---
def baixar_url(url, tentativas=3):
    for i in range(tentativas):
        try:
            r = session.get(url, timeout=20)
            if r.status_code == 200:
                return r
        except Exception:
            time.sleep(random.uniform(0.5, 2))
    return None

# --- Função para processar um produto ---
def processar_produto(produto, url_base):
    nome = achar_nome(produto)
    preco = achar_preco(produto)
    precopix = achar_precopix(produto)
    precodesconto = achar_precodesconto(produto)
    link_produto = achar_link(produto, url_base)

    brand, code = None, None
    if link_produto != "Não encontrado":
        responseprodutos = baixar_url(link_produto)
        if responseprodutos:
            soup_produto = BeautifulSoup(responseprodutos.content, 'html.parser')
            brand, code = extrair_detalhes_do_json(soup_produto)

    unit_price = limpar_preco(preco)
    discount_price = limpar_preco(precodesconto)
    pix_price = limpar_preco(precopix)

    discount = round(unit_price - discount_price, 2) if unit_price and discount_price else None
    pix_discount = round(unit_price - pix_price, 2) if unit_price and pix_price else None

    return {
        'Marca': brand,
        'GTIN': code,
        'Desconto': discount,
        'Preco com desconto': discount_price,
        'link': link_produto,
        'Nome': nome,
        'Desconto com pix': pix_discount,
        'Preco pix': pix_price,
        'Preco unitario': unit_price
    }

# --- Função para processar uma página inteira ---
def processar_pagina(pagina, url_base):
    url_pagina = f'https://www.drogariaveracruz.com.br/medicamentos/?p={pagina}'
    response = baixar_url(url_pagina)
    if not response:
        return []

    soup = BeautifulSoup(response.content, 'html.parser')
    div_produtos = soup.find_all('div', class_='li')
    resultados = []

    # processamento paralelo de produtos
    with ThreadPoolExecutor(max_workers=8) as executor:
        futures = [executor.submit(processar_produto, produto, url_base) for produto in div_produtos]
        for future in as_completed(futures):
            resultado = future.result()
            if resultado:
                resultados.append(resultado)
                print(f"Extraído (pág. {pagina}): {resultado['Nome']}")
    return resultados


# --- PRIMEIRA PÁGINA para descobrir total ---
response = baixar_url(url)
soup = BeautifulSoup(response.content, 'html.parser')
total_paginas = achar_total_paginas(soup)

lista_de_produtos = []

# --- NOVO: paralelismo também entre páginas ---
print(f"Iniciando extração em {total_paginas} páginas...\n")
with ThreadPoolExecutor(max_workers=5) as executor_paginas:  # controla quantas páginas em paralelo
    futures_paginas = [executor_paginas.submit(processar_pagina, pagina, url_base) for pagina in range(1, total_paginas + 1)]
    for future in as_completed(futures_paginas):
        resultados_pagina = future.result()
        lista_de_produtos.extend(resultados_pagina)

# --- FINAL ---
print("\nExtração concluída! Gerando tabela...")
df = pd.DataFrame(lista_de_produtos)
display(df)

df.to_csv('veracruz-final.csv', index=False)
print("Arquivo salvo: veracruzfinal.csv ✅")

Iniciando extração em 214 páginas...

Extraído (pág. 1): Renu Fresh Solução Multiuso 355ml + 120ml + Estojo para Lentes
Extraído (pág. 1): Blephagel 40g
Extraído (pág. 1): Aradois 50mg 30 comprimidos revestidos
Extraído (pág. 3): Ellura 200mg 30 cápsulas
Extraído (pág. 1): Lactosil Flora 30 cápsulas
Extraído (pág. 1): Calcitran Triflex 30 comprimidosExtraído (pág. 3): Femibion 1 28 comprimidos

Extraído (pág. 1): Materna Nestlé 30 comprimidos revestidos
Extraído (pág. 1): Salicetil Infantil 100mg 10 comprimidos
Extraído (pág. 3): Contractubex Gel 20g
Extraído (pág. 1): Opti-Free Puremoist 300ml + 120ml + Estojo de Lentes
Extraído (pág. 3): Supositório de Glicerina Granado Adulto 12un
Extraído (pág. 1): Previgrip Glóbulos Weleda 20g
Extraído (pág. 1): Teste de Gravidez Confira 1un
Extraído (pág. 3): Humulin N NPH 100UI/ml 2 Refis de 3ml cada
Extraído (pág. 1): Neosaldina Solução Oral 15ml
Extraído (pág. 3): Plenance EZE 5mg+10mg 30 Cápsulas
Extraído (pág. 1): Corus 50mg 30 comprimidos r

,Marca,GTIN,Desconto,Preco com desconto,link,Nome,Desconto com pix,Preco pix,Preco unitario
0,Biolab,7896112406617,24.60,164.63,https://www.drogariaveracruz.com.br/ellura-200...,Ellura 200mg 30 cápsulas,32.83,156.40,189.23
1,Procter&amp;Gamble,7500435185011,37.00,73.90,https://www.drogariaveracruz.com.br/suplemento...,Femibion 1 28 comprimidos,40.69,70.21,110.90
2,Biolab Sanus,7896112403418,15.39,124.51,https://www.drogariaveracruz.com.br/contractub...,Contractubex Gel 20g,21.62,118.28,139.90
3,Casa Granado,7896512901392,4.78,27.12,https://www.drogariaveracruz.com.br/supositori...,Supositório de Glicerina Granado Adulto 12un,6.14,25.76,31.90
4,Eli Lilly,7896382703317,37.87,56.80,https://www.drogariaveracruz.com.br/humulin-n-...,Humulin N NPH 100UI/ml 2 Refis de 3ml cada,40.71,53.96,94.67
...,...,...,...,...,...,...,...,...,...
4273,Boehringer,7896026301855,NaN,NaN,https://www.drogariaveracruz.com.br/cardizem-6...,Cardizem 60mg 50 comprimidos,NaN,NaN,NaN
4274,União Química,7896006201007,NaN,NaN,https://www.drogariaveracruz.com.br/urovit-200...,Urovit 200mg 18 drágeas,NaN,NaN,NaN
4275,Ache,7896658027055,NaN,NaN,https://www.drogariaveracruz.com.br/untral-25m...,"Untral 2,5mg 30 cápsulas duras",NaN,NaN,NaN
4276,Gsk,7896269900297,NaN,NaN,https://www.drogariaveracruz.com.br/betnovate-...,Betnovate 1mg/g Creme 15g,NaN,NaN,NaN


Arquivo salvo: veracruzfinal.csv ✅
